<h1> Explore and create ML datasets </h1>

このノートブックでは、New Yorkでのタクシー乗車に関するデータを探索し、乗車料金を予測する機械学習モデルを作成いたします。<br>
タクシー料金を予測する目的は、乗車後に請求に驚いたりしないように、また予想よりずっと高額な請求をされたときに抗議することができるようにすることです。

<div id="toc"></div>

では、必要なPythonライブラリのimportから始めていきましょう

In [ ]:
from google.cloud import bigquery
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shutil

<h3> BigQueryからサンプルデータを取得する</h3>


これから私達が利用するデータセットは<a href="https://bigquery.cloud.google.com/table/nyc-tlc:yellow.trips">BigQueryのpublic dataset</a>です。リンクをクリックして、カラム名をチェックしましょう。<br>
[Detail]タブに移動してレコード数が10億行あることを確認し、[Preview]タブでいくつかのデータを確認してみてください。


では、SQLを書いていくつかのフィールドをピックアップしてみましょう。

In [ ]:
sql = """
  SELECT
    pickup_datetime, pickup_longitude, pickup_latitude, dropoff_longitude,
    dropoff_latitude, passenger_count, trip_distance, tolls_amount, 
    fare_amount, total_amount 
  FROM `nyc-tlc.yellow.trips`
  LIMIT 10
"""

In [ ]:
client = bigquery.Client()
trips = client.query(sql).to_dataframe()
trips

レコード数を増やして、きれいなグラフを書いてみましょう。返されるレコードの順番は保証されないため、LIMITの数を増やしてもどのレコードが返ってくるかはわかりません。<br>
適切にデータセットを取得するために、乗車時刻のHASHを利用して10万行に1行だけを取得しましょう。つまり、10億行のデータがあれば、およそ1万行のデータが取得できます。

In [ ]:
sql = """
  SELECT
    pickup_datetime,
    pickup_longitude, pickup_latitude, 
    dropoff_longitude, dropoff_latitude,
    passenger_count,
    trip_distance,
    tolls_amount,
    fare_amount,
    total_amount
  FROM
    `nyc-tlc.yellow.trips`
  WHERE
    MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 100000) = 1
"""

In [ ]:
trips = client.query(sql).to_dataframe()
trips[:10]

<h3> データを探索する</h3>

では、データを探索し、必要に応じてクリーンアップをしていきましょう。ここでは、PythonのSeabornパッケージを利用してグラフをビジュアライズし、Pandasを使ってスライシングとフィルタリングを行います。

In [ ]:
ax = sns.regplot(x="trip_distance", y="fare_amount", fit_reg=False, ci=None, truncate=True, data=trips)
ax.figure.set_size_inches(10, 8)

なにかおかしな点があることに気づきますか？

どうやら、乗車距離が0であったり、乗車料金が明らかに非合法であったりなどの無効なデータがたくさんあるようです。<br>
これらは分析対象から除外しましょう。BigQueryへのクエリを変更して、乗車距離が0マイル以上かつ、乗車料金が最低乗車料金（$2.50）以上のデータに絞り込むことができます。

追加されたWHERE句に注目してください。

In [ ]:
sql = """
  SELECT
    pickup_datetime,
    pickup_longitude, pickup_latitude, 
    dropoff_longitude, dropoff_latitude,
    passenger_count,
    trip_distance,
    tolls_amount,
    fare_amount,
    total_amount
  FROM
    `nyc-tlc.yellow.trips`
  WHERE
    MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 100000) = 1
    AND trip_distance > 0 AND fare_amount >= 2.5
"""

In [ ]:
trips = client.query(sql).to_dataframe()
ax = sns.regplot(x="trip_distance", y="fare_amount", fit_reg=False, ci=None, truncate=True, data=trips)
ax.figure.set_size_inches(10, 8)

45ドル、50ドル付近の直線は何でしょうか？これは、たとえばマンハッタンのJFK空港、La Guardia空港からの定額料金と予測できます。
データを確認して、値が何を意味しているのかを確認してみましょう。

toll_amount（定額料金）とtotal_amount（総額料金）の関係に注目してください。

In [ ]:
tollrides = trips[trips['tolls_amount'] > 0]
tollrides[tollrides['pickup_datetime'] == pd.Timestamp('2010-04-29 12:28:00', tz = 'UTC')]

上のいくつかのサンプルデータを見てみると、総額料金(total_amount)は乗車料金(fare_amount)と定額料金(tolls_amount）を反映していることは明らかです。そして任意のチップが加えられていますが、チップを現金で支払った場合にはチップの値段はわかりません。
そのため、ここではfare_amount + tolls_amountを予測の対象として利用します。チップは乗客の裁量によるため、料金予測ツールからは除外するべきでしょう。

では、カラムごとの値の分布を確認しましょう。

In [ ]:
trips.describe()

どうやら経度(longitude)と緯度(latitude)の最小値と最大値がおかしいようです。

いくつかの履歴の乗車(pickup)位置と降車(dropoff)位置を見てみましょう。

In [ ]:
def showrides(df, numlines):
  lats = []
  lons = []
  for iter, row in df[:numlines].iterrows():
    lons.append(row['pickup_longitude'])
    lons.append(row['dropoff_longitude'])
    lons.append(None)
    lats.append(row['pickup_latitude'])
    lats.append(row['dropoff_latitude'])
    lats.append(None)

  sns.set_style("darkgrid")
  plt.figure(figsize=(10,8))
  plt.plot(lons, lats)

showrides(trips, 10)

In [ ]:
showrides(tollrides, 10)

予想したとおり、定額料金を含む乗車は他の典型的な乗車よりも距離が長いようです。

<h3> データクレンジングとその他の前処理</h3>

以下のデータクレンジングが必要です。

<ol>
<li>New Yorkの経度は-74前後、緯度は41前後</li>
<li>乗客が0人のデータは含めない</li>
<li>総額料金(total_amount)を、乗車料金(fare_amount)と定額料金(tolls_amount)だけを反映するようにする。またこれら２つの列は除外する</li>
<li>乗車時には、乗車位置と降車位置はわかっていますが、乗車距離はわかりません（乗車距離はルートによって変わるため）。そのため乗車距離は機械学習のデータセットには含めない</li>
<li>タイムスタンプを削除する</li>
</ol>

距離0の乗車を除外したのと同様にBigQueryを使って前処理を行うことができますが、他の方法をお見せするためにここではPythonを使いましょう。<br>
本番環境では、リアルタイムの入力データに対して同様の前処理を行う必要があります。（Dataflowなどを利用する）

入力データに対するこのような前処理は、機械学習を行う際に一般的です。

In [ ]:
def preprocess(trips_in):
  trips = trips_in.copy(deep=True)
  trips.fare_amount = trips.fare_amount + trips.tolls_amount
  del trips['tolls_amount']
  del trips['total_amount']
  del trips['trip_distance']
  del trips['pickup_datetime']
  qc = np.all([\
             trips['pickup_longitude'] > -78, \
             trips['pickup_longitude'] < -70, \
             trips['dropoff_longitude'] > -78, \
             trips['dropoff_longitude'] < -70, \
             trips['pickup_latitude'] > 37, \
             trips['pickup_latitude'] < 45, \
             trips['dropoff_latitude'] > 37, \
             trips['dropoff_latitude'] < 45, \
             trips['passenger_count'] > 0,
            ], axis=0)
  return trips[qc]

tripsqc = preprocess(trips)
tripsqc.describe()

データクレンジングにより、およそ300行(11400 - 11101)、全体の3％ほどのデータが除外されました。これは良いバランスです。

では、機械学習用データセットを作成しましょう。

<h3> 機械学習用データセットの作成 </h3>

クレンジングされたデータを学習用と検証用、テスト用のデータセットにランダムに分割しましょう。

In [ ]:
shuffled = tripsqc.sample(frac=1)
trainsize = int(len(shuffled['fare_amount']) * 0.70)
validsize = int(len(shuffled['fare_amount']) * 0.15)

df_train = shuffled.iloc[:trainsize, :]
df_valid = shuffled.iloc[trainsize:(trainsize+validsize), :]
df_test = shuffled.iloc[(trainsize+validsize):, :]

In [ ]:
df_train.describe()

In [ ]:
df_valid.describe()

In [ ]:
df_test.describe()

3つのDataframeをcsvファイルに書き出しましょう。<br>
DataflowやCloud MLを利用できるようになるまでは、ローカルでのトレーニングにこれらのファイルを使用します（これらのファイルは全データの内の1/100000件のデータのみを含むということを思い出してください）

In [ ]:
def to_csv(df, filename):
  outdf = df.copy(deep=False)
  outdf.loc[:, 'key'] = np.arange(0, len(outdf)) # rownumber as key
  # reorder columns so that target is first column
  cols = outdf.columns.tolist()
  cols.remove('fare_amount')
  cols.insert(0, 'fare_amount')
  print (cols)  # new order of columns
  outdf = outdf[cols]
  outdf.to_csv(filename, header=False, index_label=False, index=False)

to_csv(df_train, 'taxi-train.csv')
to_csv(df_valid, 'taxi-valid.csv')
to_csv(df_test, 'taxi-test.csv')

In [ ]:
!head -10 taxi-valid.csv

<h3> データセットが存在することを確認する </h3>

In [ ]:
!ls -l *.csv

学習、検証、テストに対応する３つのcsvファイルが作成されました。ファイルサイズの割合はデータの分割の割合に対応しています。

In [ ]:
%%bash
head taxi-train.csv

上手くいっていますね！機械学習用のデータセットが作成され、機械学習モデルのトレーニング、検証、評価の準備ができました。

<h3> ベンチマーク </h3>

複雑な機械学習モデルを作成するまえに、簡単なモデルを作成してそれをベンチマークとするのは良いアイデアです。

以下のモデルは、総額料金(fare_amount)の平均を乗車距離(trip_distance)で割り、距離毎の料金の割合を計算します。<br>
これを用いてRMSEを算出してみましょう。

In [ ]:
def distance_between(lat1, lon1, lat2, lon2):
  # haversine formula to compute distance "as the crow flies".  Taxis can't fly of course.
  dist = np.degrees(np.arccos(np.minimum(1,np.sin(np.radians(lat1)) * np.sin(np.radians(lat2)) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.cos(np.radians(lon2 - lon1))))) * 60 * 1.515 * 1.609344
  return dist

def estimate_distance(df):
  return distance_between(df['pickuplat'], df['pickuplon'], df['dropofflat'], df['dropofflon'])

def compute_rmse(actual, predicted):
  return np.sqrt(np.mean((actual-predicted)**2))

def print_rmse(df, rate, name):
  print ("{1} RMSE = {0}".format(compute_rmse(df['fare_amount'], rate*estimate_distance(df)), name))

FEATURES = ['pickuplon','pickuplat','dropofflon','dropofflat','passengers']
TARGET = 'fare_amount'
columns = list([TARGET])
columns.extend(FEATURES) # in CSV, target is the first column, after the features
columns.append('key')
df_train = pd.read_csv('taxi-train.csv', header=None, names=columns)
df_valid = pd.read_csv('taxi-valid.csv', header=None, names=columns)
df_test = pd.read_csv('taxi-test.csv', header=None, names=columns)
rate = df_train['fare_amount'].mean() / estimate_distance(df_train).mean()
print ("Rate = ${0}/km".format(rate))
print_rmse(df_train, rate, 'Train')
print_rmse(df_valid, rate, 'Valid') 
print_rmse(df_test, rate, 'Test') 

<h2>同じデータセットのベンチマーク</h2>

RMSEはデータセットに依存するため、比較のために同じデータセットを評価に利用します。<br>
以下のクエリーは後のラボでも使用します。

In [ ]:
def create_query(phase, EVERY_N):
  """
  phase: 1=train 2=valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key,
  EXTRACT(DAYOFWEEK FROM pickup_datetime)*1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime)*1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # training
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) < 2".format(base_query)
    else:
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) = {1}".format(base_query, phase)
  else:
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), {1}) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df_valid = client.query(query).to_dataframe()
print_rmse(df_valid, 2.56, 'Final Validation Set')

簡単な距離ベースのルールではRMSEが<b>7.42ドル</b>です。もちろん、この値を超えなければなりませんが、このような簡単なルールを超えるのは驚くほど難しいものです。

しかし、まずは野心的に、テストデータで6ドル以下のRMSEとなる機械学習モデルを作成することを目指していきましょう。


Copyright 2016 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.